In [ ]:
import sklearn.metrics
from sklearn.metrics import confusion_matrix

from src.Biologging_Toolkit.utils.acoustic_utils import sort_spectrogram
%load_ext autoreload
%autoreload 2
import sys
import os
import numpy as np
import random
import matplotlib.pyplot as plt
sys.path.append('../src/')
from Biologging_Toolkit.applications.Drift_Dives import DriftDives
from Biologging_Toolkit.plot.dives import *

In [ ]:
depids = ['ml17_280a','ml18_296a','ml18_294b','ml19_292a','ml19_292b','ml19_293a','ml19_294a','ml20_293a','ml20_296b','ml20_313a','ml21_295a','ml21_305b']
depid = depids[5]
path = f'D:/individus_brut/individus/{depid}/'
acoustic_path = os.path.join(path, 'dives')
sens_path = f'D:/individus_brut/CTD/{depid}/{depid}sens5.nc'

In [ ]:
depid

### Get drift dives


In [ ]:
inst = DriftDives(depid,
            path = path,
            sens_path = sens_path
            )

In [ ]:
inst(mode='inertial',overwrite = True) 

 #### Choose method to identify drift dives ('inertial', 'depth')

In [ ]:
inst.acoustic_cluster(min_cluster_size = 10, min_samples = 3, computed = True)

In [ ]:
inst.acoustic_cluster(acoustic_path=os.path.join(path, 'dives'),
                      #freqs = [0, 16000],
                      sort = True, tmin = 0, tmax = 14, timestep = 1,
                      freq_sampling = 'linear', nfeatures = 513, min_cluster_size=20, min_samples=13)

### SEE CLUSTERING RESULTS

In [ ]:
 plot_clusters(inst, save = False, save_path = 'C:/Users/grosm/Desktop/thèse/Figures')

In [ ]:
  inst.save_cluster(cluster = [1, 2, 3, 4, 6], overwrite = True, **{'nfeature':513, 'type':'linear','tmin':0,'tmax':15,'timestep':1})

In [ ]:
inst.compute_metric(cluster = [0,1,2,3,4,6])

In [ ]:
dives = inst.ds['dives'][:].data
acc_drifts = inst.ds['acc_drift'][:].data
inertial_drifts = inst.ds['inertial_drift'][:].data
cluster_drifts = inst.ds['cluster_drifts'][:].data
cluster_drifts[dives == 455]
#np.all(acc_drifts[dives == 455] == 0)

In [ ]:
np.unique(cluster_drifts)

### Threshold

In [ ]:
thresholds = [-15, -10, -5]
acoustic_drift = []
for thresh in thresholds :
    inst.acoustic_feature_threshold(frequency = list(range(30,45)), acoustic_path = acoustic_path, threshold = thresh)
    acoustic_drift.append(inst.acoustic_drifts)

In [ ]:
acc_drift = inst.ds['acc_drift'][:].data
depth_drift = inst.ds['depth_drift'][:].data
dives = inst.ds['dives'][:].data
inertial = inst.ds['inertial_drift'][:].data
unique_dives = np.unique(dives)

In [ ]:
acc_dive, depth_dive, inert_dive, thres1, thres2, thres3 = [],[],[],[],[], []
for dive in unique_dives:
    acc_dive.append(np.any(acc_drift[dives == dive] == 1))
    depth_dive.append(np.any(depth_drift[dives == dive] == 1))
    inert_dive.append(np.any(inertial[dives == dive] == 1))
    '''thres1.append(np.any(acoustic_drift[0][dives == dive] == 1))
    thres2.append(np.any(acoustic_drift[1][dives == dive] == 1))
    thres3.append(np.any(acoustic_drift[2][dives == dive] == 1))'''

In [ ]:
label = np.array(acc_dive) & np.array(depth_dive)
print(confusion_matrix(label, acc_dive))
print(confusion_matrix(label, depth_dive))
print(confusion_matrix(label, inert_dive))
'''print(confusion_matrix(label, thres1))
print(confusion_matrix(label, thres2))
print(confusion_matrix(label, thres3))'''

### Check results

In [ ]:
for depid in depids :
    path = f'D:/individus_brut/individus/{depid}/'
    acoustic_path = os.path.join(path, 'dives')
    sens_path = f'D:/individus_brut/CTD/{depid}/{depid}sens5.nc'
    inst = DriftDives(depid,
            path = path,
            sens_path = sens_path
            )
    acc_drift = inst.ds['acc_drift'][:].data
    depth_drift = inst.ds['depth_drift'][:].data
    dives = inst.ds['dives'][:].data
    acoustic = inst.ds['cluster_drifts'][:].data
    acc_dive, depth_dive, inert_dive, clust_dive = [],[],[],[]
    for dive in unique_dives:
        acc_dive.append(np.any(acc_drift[dives == dive] == 1))
        depth_dive.append(np.any(depth_drift[dives == dive] == 1))
        clust_dive.append(np.any(acoustic[dives == dive] == 1))
    print(depid)
    label = np.array(acc_dive) & np.array(depth_dive)
    print(confusion_matrix(label, clust_dive))


In [ ]:
depid

In [ ]:
dives = inst.ds['dives'][:].data
cluster = inst.ds['cluster_drifts'][:].data
bank_angle = inst.ds['bank_angle'][:].data
depths = inst.ds['depth'][:].data
unique_dives = np.unique(dives)
label = []
count = 0
for dive in unique_dives :
    _cluster = cluster[dives == dive][0]
    if _cluster == 1 :
        count += 1
        fig, ax = plt.subplots(1, 2)
        ax1 = ax[0].twinx()
        ax[0].plot(depths[dives == dive])
        ax1.scatter(np.arange(0, len(bank_angle[dives == dive])),
                                bank_angle[dives == dive], s = 2, c = 'orange')
        ax[1].imshow(np.load(f'D:/individus_brut/individus/{depid}/dives/acoustic_dive_{int(dive):05}.npz')['spectro'].T, origin = 'lower')
        fig.show()
print(count)

 ### SHOW SPECTROGRAMS FROM ONE CLUSTER

In [ ]:
cluster = 0
drift = inst.cluster_fns[labels == 1]
fig, ax = plt.subplots(3,3, figsize = (15,15))
ax = ax.flatten()
random.shuffle(drift)
for i, _drift in enumerate(drift[:9]) :
    ax[i].imshow(np.load(_drift)['spectro'].T, origin = 'lower')

In [ ]:
cluster = 0
inst.save_cluster(cluster = cluster)

In [ ]:
inst.ds.close()

### RUN ALL SES

In [ ]:
ground_truth = []
preds = []
for depid in depids[:-1] :
    path = f'D:/individus_brut/individus/{depid}/'
    sens_path = f'D:/individus_brut/CTD/{depid}/{depid}sens5.nc'
    inst = DriftDives(depid,
                      path=path,
                      sens_path=sens_path)
    _ground_truth = inst.ds['depth_drift'][:].data.astype(bool) & inst.ds['acc_drift'][:].data.astype(bool)
    ground_truth.extend(_ground_truth.astype(int))
    preds.extend(inst.ds['cluster_drifts'][:].data if 'cluster_drifts' in inst.ds.variables.keys() else np.full(len(ground_truth), np.nan))

In [ ]:
import sklearn
sklearn.metrics.confusion_matrix(ground_truth, preds)

In [ ]:
import umap
import hdbscan
labels = []
X = []
for depid in depids:
    path = f'D:/individus_brut/individus/{depid}/'
    sens_path = f'D:/individus_brut/CTD/{depid}/{depid}sens5.nc'
    inst = DriftDives(depid,
                      path=path,
                      analysis_length=60,
                      sens_path=sens_path)
    inst.acoustic_cluster(acoustic_path=os.path.join(path, 'dives'))
    X.extend(inst.X)
    acc_drifts  = inst.ds['acc_drift'][:].data
    dives = inst.ds['dives'][:].data
    for fn in inst.cluster_fns :
        dive = int(fn.split('.')[0][-4:])
        if np.all(acc_drifts[dives == dive] == 0) == False :
            labels.append(1)
        else :
            labels.append(0)

In [ ]:
project = umap.UMAP()
embed = project.fit_transform(X)
clusterer = hdbscan.HDBSCAN(min_cluster_size=50, min_samples=10).fit(embed)
fig, ax = plt.subplots(1,2, figsize = (15,8))
scatter = ax[0].scatter(embed[:,0], embed[:,1], c=labels, s = 1)
for label in np.unique(labels):
    ax[0].scatter([], [], c=scatter.cmap(scatter.norm(label)), label=f'Cluster {label}')
ax[0].legend(title="Clusters")
scatter = ax[1].scatter(embed[:,0], embed[:,1], c=clusterer.labels_, s = 1)
for label in np.unique(clusterer.labels_):
    ax[1].scatter([], [], c=scatter.cmap(scatter.norm(label)), label=f'Cluster {label}')
ax[1].legend(title="Clusters")
fig.show()

In [ ]:
import netCDF4 as nc
import pandas as pd
from scipy.interpolate import interp1d
for depid in depids :
    path = f'D:/individus_brut/individus/{depid}/'
    ds = nc.Dataset(os.path.join(path, f'{depid}_sens.nc'))
    df = pd.read_csv(os.path.join(path, f'{depid}_dive.csv'))
    timestamps = ds['time'][:].data
    lat_interp = interp1d(timestamps, ds['lat'][:].data)
    lon_interp = interp1d(timestamps, ds['lon'][:].data)
    acc = ds['acc_drift'][:].data
    dives = ds['dives'][:].data
    cluster_drifts = ds['cluster_drifts'][:].data
    df['end_lat'] = lat_interp(df.end_time)
    df['end_lon'] = lon_interp(df.end_time)
    _acc = []
    _clust = []
    for dive in np.unique(dives) :
        if np.all(acc[dives == dive] == 0) == False :
            _acc.append(1)
        else :
            _acc.append(0)
        if np.all(cluster_drifts[dives == dive] == 0) == False :
            _clust.append(0)
        else :
            _clust.append(1)
    df['acc_drifts'] = _acc
    df['cluster_drifts'] = _clust
    df.to_csv(os.path.join(path, f'{depid}_dive.csv'), index = None)

In [ ]:
import netCDF4 as nc
import pandas as pd
from scipy.interpolate import interp1d
for depid in depids :
    path = f'D:/individus_brut/individus/{depid}/'
    df = pd.read_csv(os.path.join(path, f'{depid}_dive.csv'))
    print(np.isin(np.where(df.cluster_drifts == 1), np.where(df.acc_drifts == 1)).sum()/len(df[df.cluster_drifts == 1]))